# Import data

Remark: The 'week' column report the Monday of the considered week. 

In [64]:
import pandas as pd
sales=pd.read_csv('data_raw.csv',parse_dates=['week'])
sales

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality
0,2016-10-31,1,135.0,True,black,10.16,6,06.Mobile phone accessories
1,2016-11-07,1,102.0,True,black,9.86,6,06.Mobile phone accessories
2,2016-11-14,1,110.0,True,black,10.24,6,06.Mobile phone accessories
3,2016-11-21,1,127.0,True,black,8.27,6,06.Mobile phone accessories
4,2016-11-28,1,84.0,True,black,8.83,6,06.Mobile phone accessories
...,...,...,...,...,...,...,...,...
4395,2018-08-27,44,20.0,False,black,53.99,6,09.Smartphone stands
4396,2018-09-03,44,14.0,False,NaN,52.99,6,09.Smartphone stands
4397,2018-09-10,44,22.0,True,black,44.99,6,09.Smartphone stands
4398,2018-09-17,44,28.0,True,NaN,42.99,6,09.Smartphone stands


In [65]:
sales.sku.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44])

In [66]:
sales[['price','weekly_sales']].describe()

,price,weekly_sales
count,4400.000000,4400.000000
mean,44.432709,83.054773
std,42.500295,288.000205
min,2.390000,0.000000
25%,15.680000,11.000000
50%,27.550000,25.000000
75%,54.990000,70.000000
max,227.720000,7512.000000


# I/ Data Processing and factor modeling

## 1) Missing Data

In [67]:
sales.isna().any()

week              False
sku               False
weekly_sales      False
feat_main_page    False
color              True
price             False
vendor            False
functionality     False
dtype: bool

In [68]:
sales.dropna(inplace= False)  #replace by True to apply the deletion within the dataset

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality
0,2016-10-31,1,135.0,True,black,10.16,6,06.Mobile phone accessories
1,2016-11-07,1,102.0,True,black,9.86,6,06.Mobile phone accessories
2,2016-11-14,1,110.0,True,black,10.24,6,06.Mobile phone accessories
3,2016-11-21,1,127.0,True,black,8.27,6,06.Mobile phone accessories
4,2016-11-28,1,84.0,True,black,8.83,6,06.Mobile phone accessories
...,...,...,...,...,...,...,...,...
4393,2018-08-13,44,52.0,True,black,43.99,6,09.Smartphone stands
4394,2018-08-20,44,29.0,True,black,42.38,6,09.Smartphone stands
4395,2018-08-27,44,20.0,False,black,53.99,6,09.Smartphone stands
4397,2018-09-10,44,22.0,True,black,44.99,6,09.Smartphone stands


In [69]:
sales[sales['color'].isnull()]

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality
802,2016-11-14,9,54.0,True,NaN,139.44,9,11.Fitness trackers
803,2016-11-21,9,71.0,True,NaN,141.16,9,11.Fitness trackers
4133,2017-06-19,42,4.0,False,NaN,27.33,10,09.Smartphone stands
4196,2018-09-03,42,8.0,False,NaN,42.99,10,09.Smartphone stands
4197,2018-09-10,42,14.0,True,NaN,42.99,10,09.Smartphone stands
4200,2016-10-31,43,5.0,True,NaN,109.99,9,11.Fitness trackers
4314,2017-02-06,44,5.0,False,NaN,53.99,6,09.Smartphone stands
4391,2018-07-30,44,34.0,True,NaN,41.99,6,09.Smartphone stands
4396,2018-09-03,44,14.0,False,NaN,52.99,6,09.Smartphone stands
4398,2018-09-17,44,28.0,True,NaN,42.99,6,09.Smartphone stands


In [70]:
sales[sales.sku ==44]["color"].value_counts(dropna=False)

black    96
NaN       4
Name: color, dtype: int64

In [71]:
sales[sales.sku ==43]["color"].value_counts(dropna=False)

gold    99
NaN      1
Name: color, dtype: int64

In [72]:
sales[sales.sku ==42]["color"].value_counts(dropna=False)

black    97
NaN       3
Name: color, dtype: int64

In [73]:
sales[sales.sku ==9]["color"].value_counts(dropna=False)

black    98
NaN       2
Name: color, dtype: int64

For one item (SKU 44), we do manual imputation:

In [74]:
# Manual imputation
# sales.at[1, 'color']= "black" 
# sales.at[3, 'color']= "black" 
# sales.at[8, 'color']= "black" 
# sales.at[85, 'color']= "black" 

For the other item, we introduce sklearn imputer:

In [75]:
# Using sklearn imputer
import numpy as np 
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

SKU 43:

In [76]:
missing_idx_43 = sales[(sales.color.isna()) & (sales.sku==43)].index.values
missing_idx_43

array([4200])

In [77]:
imputer.fit(sales[sales.sku==43][["sku","color"]])
for i in missing_idx_43:
  sales.at[i, 'color']= imputer.transform(sales[ (sales.sku ==43) & (sales.color.isna())][["sku","color"]] )[0,1]

SKUs 44,42 and 9:

In [78]:
missing_idx_42 = sales[(sales.color.isna()) & (sales.sku==42)].index.values
imputer.fit(sales[sales.sku==42][["sku","color"]])
for i in missing_idx_42:
  sales.at[i, 'color']= imputer.transform(sales[ (sales.sku ==42) & (sales.color.isna())][["sku","color"]] )[0,1]

missing_idx_44 = sales[(sales.color.isna()) & (sales.sku==44)].index.values
print(missing_idx_44)
imputer.fit(sales[sales.sku==44][["sku","color"]])
for i in missing_idx_44:
  sales.at[i, 'color']= imputer.transform(sales[ (sales.sku ==44) & (sales.color.isna())][["sku","color"]] )[0,1]

missing_idx_9 = sales[(sales.color.isna()) & (sales.sku==9)].index.values
imputer.fit(sales[sales.sku==9][["sku","color"]])
for i in missing_idx_9:
  sales.at[i, 'color']= imputer.transform(sales[ (sales.sku ==9) & (sales.color.isna())][["sku","color"]] )[0,1]


[4314 4391 4396 4398]


In [79]:
sales[sales['color'].isnull()]

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality


In [80]:
def check_outliers(df,features,k=5):
  data = df.copy()
  for f in features:
    # data['mean+'+str(k)+'*std_'+f] = data.groupby('sku')[f].transform(
    # lambda x: x.mean()+k*x.std()  )
    # data['mean-'+str(k)+'*std_'+f] = data.groupby('sku')[f].transform(
    # lambda x: x.mean()-k*x.std()  )
    data['outlier_'+f] = data.groupby('sku')[f].transform(
    lambda x: (x > (x.mean()+k*x.std()) )  |(x < (x.mean()-k*x.std()) )  )
  return(data)

In [81]:
df = check_outliers(sales,['price','weekly_sales'],5)

In [82]:
df[df.outlier_price]

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality,outlier_price,outlier_weekly_sales
927,2017-05-08,10,9.0,True,white,130.89,9,10.VR headset,True,False
1105,2016-12-05,12,8.0,False,black,135.91,6,01.Streaming sticks,True,False
2863,2018-01-15,29,11.0,False,grey,170.76,6,06.Mobile phone accessories,True,False
3992,2018-08-06,40,51.0,False,black,33.08,5,06.Mobile phone accessories,True,False
4132,2017-06-12,42,3.0,False,black,87.98,10,09.Smartphone stands,True,False
4376,2018-04-16,44,2.0,True,black,112.83,6,09.Smartphone stands,True,False


In [83]:
df[df.outlier_weekly_sales]

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality,outlier_price,outlier_weekly_sales
506,2016-12-12,6,119.0,True,blue,17.10,3,04.Selfie sticks,False,True
699,2018-09-24,7,724.0,False,blue,6.26,3,04.Selfie sticks,False,True
991,2018-07-30,10,75.0,True,white,189.70,9,10.VR headset,False,True
1110,2017-01-09,12,750.0,True,black,32.01,6,01.Streaming sticks,False,True
1145,2017-09-11,12,579.0,True,black,31.96,6,01.Streaming sticks,False,True
1233,2017-06-19,13,63.0,True,black,20.99,10,09.Smartphone stands,False,True
1548,2017-10-02,16,1027.0,True,blue,11.10,8,02.Portable smartphone chargers,False,True
1623,2017-04-10,17,318.0,True,blue,20.04,6,06.Mobile phone accessories,False,True
1722,2017-04-03,18,51.0,True,black,93.71,6,08.Digital pencils,False,True
1877,2018-04-23,19,853.0,True,black,64.99,6,01.Streaming sticks,False,True


## 2) Time feature enginerring

In [84]:
sales.head()

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality
0,2016-10-31,1,135.0,True,black,10.16,6,06.Mobile phone accessories
1,2016-11-07,1,102.0,True,black,9.86,6,06.Mobile phone accessories
2,2016-11-14,1,110.0,True,black,10.24,6,06.Mobile phone accessories
3,2016-11-21,1,127.0,True,black,8.27,6,06.Mobile phone accessories
4,2016-11-28,1,84.0,True,black,8.83,6,06.Mobile phone accessories


In [85]:
sales['trend'] = sales['week'].dt.year - 2016
sales['month'] = sales['week'].dt.month #We consider that a week belongs to the month of the Monday 
sales.head()

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality,trend,month
0,2016-10-31,1,135.0,True,black,10.16,6,06.Mobile phone accessories,0,10
1,2016-11-07,1,102.0,True,black,9.86,6,06.Mobile phone accessories,0,11
2,2016-11-14,1,110.0,True,black,10.24,6,06.Mobile phone accessories,0,11
3,2016-11-21,1,127.0,True,black,8.27,6,06.Mobile phone accessories,0,11
4,2016-11-28,1,84.0,True,black,8.83,6,06.Mobile phone accessories,0,11


In [86]:
sales.loc[0]

week                      2016-10-31 00:00:00
sku                                         1
weekly_sales                              135
feat_main_page                           True
color                                   black
price                                   10.16
vendor                                      6
functionality     06.Mobile phone accessories
trend                                       0
month                                      10
Name: 0, dtype: object

In [87]:
sales = pd.get_dummies(data=sales, columns=['month'], drop_first = True)

In [88]:
sales.loc[0]

week                      2016-10-31 00:00:00
sku                                         1
weekly_sales                              135
feat_main_page                           True
color                                   black
price                                   10.16
vendor                                      6
functionality     06.Mobile phone accessories
trend                                       0
month_2                                     0
month_3                                     0
month_4                                     0
month_5                                     0
month_6                                     0
month_7                                     0
month_8                                     0
month_9                                     0
month_10                                    1
month_11                                    0
month_12                                    0
Name: 0, dtype: object

In [89]:
sales

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2016-10-31,1,135.0,True,black,10.16,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,1,0,0
1,2016-11-07,1,102.0,True,black,9.86,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0
2,2016-11-14,1,110.0,True,black,10.24,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0
3,2016-11-21,1,127.0,True,black,8.27,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0
4,2016-11-28,1,84.0,True,black,8.83,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,2018-08-27,44,20.0,False,black,53.99,6,09.Smartphone stands,2,0,0,0,0,0,0,1,0,0,0,0
4396,2018-09-03,44,14.0,False,black,52.99,6,09.Smartphone stands,2,0,0,0,0,0,0,0,1,0,0,0
4397,2018-09-10,44,22.0,True,black,44.99,6,09.Smartphone stands,2,0,0,0,0,0,0,0,1,0,0,0
4398,2018-09-17,44,28.0,True,black,42.99,6,09.Smartphone stands,2,0,0,0,0,0,0,0,1,0,0,0


## 3) Price

In [90]:
## Lag prices
sales["price-1"] = sales.groupby(['sku'])['price'].shift(1)
sales["price-2"] = sales.groupby(['sku'])['price'].shift(2)
sales.dropna(subset=['price-1',"price-2"],inplace=True)
sales.head()

,week,sku,weekly_sales,feat_main_page,color,price,vendor,functionality,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,price-1,price-2
2,2016-11-14,1,110.0,True,black,10.24,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0,9.86,10.16
3,2016-11-21,1,127.0,True,black,8.27,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0,10.24,9.86
4,2016-11-28,1,84.0,True,black,8.83,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0,8.27,10.24
5,2016-12-05,1,87.0,True,black,8.98,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,0,1,8.83,8.27
6,2016-12-12,1,64.0,True,black,10.40,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,0,1,8.98,8.83


In [91]:
## Put lag-prices next to the price column ##
#price
col = sales.pop("price")
sales.insert(3, col.name, col)
pos_price=sales.columns.get_loc('price')
#p-1
col = sales.pop("price-1")
sales.insert(pos_price+1, col.name, col)
#p-2
col = sales.pop("price-2")
sales.insert(pos_price+2, col.name, col)
#plot
sales.head()

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,color,vendor,functionality,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
2,2016-11-14,1,110.0,10.24,9.86,10.16,True,black,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0
3,2016-11-21,1,127.0,8.27,10.24,9.86,True,black,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0
4,2016-11-28,1,84.0,8.83,8.27,10.24,True,black,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,1,0
5,2016-12-05,1,87.0,8.98,8.83,8.27,True,black,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,0,1
6,2016-12-12,1,64.0,10.40,8.98,8.83,True,black,6,06.Mobile phone accessories,0,0,0,0,0,0,0,0,0,0,0,1


## 4) Feature on main page

In [92]:
## feat_main_page
sales['feat_main_page'] = sales.feat_main_page.astype('int')

## 5) Item descriptive feature

In [93]:
sales = pd.get_dummies(data=sales, columns=['functionality','color','vendor'], drop_first = True)
sales.head()

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,functionality_02.Portable smartphone chargers,functionality_03.Bluetooth speakers,functionality_04.Selfie sticks,functionality_05.Bluetooth tracker,functionality_06.Mobile phone accessories,functionality_07.Headphones,functionality_08.Digital pencils,functionality_09.Smartphone stands,functionality_10.VR headset,functionality_11.Fitness trackers,functionality_12.Flash drives,color_blue,color_gold,color_green,color_grey,color_none,color_pink,color_purple,color_red,color_white,vendor_2,vendor_3,vendor_4,vendor_5,vendor_6,vendor_7,vendor_8,vendor_9,vendor_10
2,2016-11-14,1,110.0,10.24,9.86,10.16,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2016-11-21,1,127.0,8.27,10.24,9.86,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2016-11-28,1,84.0,8.83,8.27,10.24,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,2016-12-05,1,87.0,8.98,8.83,8.27,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,2016-12-12,1,64.0,10.40,8.98,8.83,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


## 6) Export dataset

In [94]:
sales = sales.sort_values(by=['sku','week'])
sales

,week,sku,weekly_sales,price,price-1,price-2,feat_main_page,trend,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,functionality_02.Portable smartphone chargers,functionality_03.Bluetooth speakers,functionality_04.Selfie sticks,functionality_05.Bluetooth tracker,functionality_06.Mobile phone accessories,functionality_07.Headphones,functionality_08.Digital pencils,functionality_09.Smartphone stands,functionality_10.VR headset,functionality_11.Fitness trackers,functionality_12.Flash drives,color_blue,color_gold,color_green,color_grey,color_none,color_pink,color_purple,color_red,color_white,vendor_2,vendor_3,vendor_4,vendor_5,vendor_6,vendor_7,vendor_8,vendor_9,vendor_10
2,2016-11-14,1,110.0,10.24,9.86,10.16,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2016-11-21,1,127.0,8.27,10.24,9.86,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2016-11-28,1,84.0,8.83,8.27,10.24,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
5,2016-12-05,1,87.0,8.98,8.83,8.27,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,2016-12-12,1,64.0,10.40,8.98,8.83,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4395,2018-08-27,44,20.0,53.99,42.38,43.99,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4396,2018-09-03,44,14.0,52.99,53.99,42.38,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4397,2018-09-10,44,22.0,44.99,52.99,53.99,1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4398,2018-09-17,44,28.0,42.99,44.99,52.99,1,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [95]:
sales.to_csv('data_processed.csv',index=False)